<a href="https://colab.research.google.com/github/MicroprocessorX069/Handwriting-comparison/blob/master/LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import math

In [0]:
#Dataset=pd.read_csv('Data_equalSamples (1).csv')
Dataset=pd.read_csv('Data.csv')

In [0]:
def createDataset(df,train_percent, validation_percent):
  df=df.loc[:,'Af1':'target']
  length=df.shape[0]
  
  trainEnd=int(length*train_percent)
  
  validationEnd=trainEnd+int(length*validation_percent)
  
  trainData=df.iloc[0:trainEnd]
  trainResult=trainData.loc[:,'target']
  trainData=trainData.loc[:,'Af1':'Bf9']
  trainResult=trainResult.reset_index(drop=True)
  trainData=trainData.reset_index(drop=True)
  
  validationData=df.iloc[trainEnd:validationEnd]
  validationResult=validationData.loc[:,'target']
  validationData=validationData.loc[:,'Af1':'Bf9']
  validationResult=validationResult.reset_index(drop=True)
  validationData=validationData.reset_index(drop=True)
  
  testData=df.iloc[validationEnd:length]
  testResult=testData.loc[:,'target']
  testData=testData.loc[:,'Af1':'Bf9']
  testResult=testResult.reset_index(drop=True)
  testData=testData.reset_index(drop=True)
  
  return trainData, trainResult, validationData, validationResult, testData, testResult
  
 

In [0]:
validation_percent=0
train_percent=0.7

trainData, trainResult, validationData, validationResult, testData, testResult = createDataset(Dataset,train_percent, validation_percent)

trainData=np.array(trainData)
trainResult=np.array(trainResult)
testData=np.array(testData)
testResult=np.array(testResult)


In [29]:
trainData[7]

array([2, 1, 1, 0, 2, 2, 0, 2, 2, 2, 1, 1, 3, 2, 2, 1, 0, 2])

In [58]:
import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))
W=np.zeros(19)
j=-1
learning_rate=0.2
for epoch in range(0,10):
  j=-1
  for input in trainData:
    output=W[0]
    j=j+1

    for i in range(1,len(W)):
      output=output+W[i]*input[i-1]
    target=trainResult[j]
    output=sigmoid(output)
    for i in range(0,len(W)):  
      if(i==0):
        weight_change=learning_rate*(target-output)*output*(1-output)
        W[i]=W[i]+weight_change
      else:
        weight_change=learning_rate*(target-output)*output*(1-output)*input[i-1]
        W[i]=W[i]+weight_change

print(j)

205675


In [59]:
W

array([-0.44371703, -0.19046304, -0.13523363, -0.37375809, -0.18336227,
       -0.28781533, -0.57090302, -0.33069431, -0.21364701, -0.44928934,
       -0.0586004 , -0.1682542 , -0.22918535, -0.16387121,  0.03391098,
       -0.37033542, -0.29752859, -0.06429286, -0.35325656])

In [61]:
correct=0
j=-1
for input in testData:
  output=W[0]
  j=j+1
  for i in range(1,len(W)):
    output=output+W[i]*input[i-1]
  target=testResult[j]
  output=round(sigmoid(output),0)
  if(target==output):
    correct=correct+1
print((correct*100)/j)

99.72885893857918


In [48]:
testResult.shape

(475,)